# Data Processing

For the project:  
temperature data, daily mean temperatures for the weather station Berlin Tempelhof!  

### important notes
The data set is in a text file "ECA_blended_custom.txt"  
Check for the missing values  
Think about imputation method  
Think about Train-test split   

In [1]:
# data analysis packages
import pandas as pd
import numpy as np

# timestamp manipulation 
from dateutil.relativedelta import relativedelta

### Load Data

In [2]:
# path of data file
FILE_PATH="./data_temp_raw.txt"

df=pd.read_csv(
    FILE_PATH,
    # datastarts from line 20
    skiprows=19,
    # remove empty space in column name
    skipinitialspace=True,
    # parse DATE coulumn
    parse_dates=[1])

In [3]:
df.head()


,SOUID,DATE,TG,Q_TG
0,127488,1876-01-01,22,0
1,127488,1876-01-02,25,0
2,127488,1876-01-03,3,0
3,127488,1876-01-04,-58,0
4,127488,1876-01-05,-98,0


In [4]:
df.tail()

,SOUID,DATE,TG,Q_TG
53686,111448,2022-12-27,42,0
53687,111448,2022-12-28,63,0
53688,111448,2022-12-29,100,0
53689,111448,2022-12-30,76,0
53690,111448,2022-12-31,148,0


### missing values

In [5]:
# filter dataframe for days with missing value
df_missing = df[df['Q_TG']==9]

In [6]:
df_missing.head()

,SOUID,DATE,TG,Q_TG
25316,127488,1945-04-25,-9999,9
25317,127488,1945-04-26,-9999,9
25318,127488,1945-04-27,-9999,9
25319,127488,1945-04-28,-9999,9
25320,127488,1945-04-29,-9999,9


In [7]:
# first date with missing value
df_missing['DATE'].min()

Timestamp('1945-04-25 00:00:00')

In [8]:
# last date with missing value
df_missing['DATE'].max()

Timestamp('1945-11-05 00:00:00')

In [9]:
# number of days with missing values
missing_days = (df_missing['DATE'].max()-df_missing['DATE'].min()).days + 1
missing_days

195

In [10]:
# check if all days in between have missing values
missing_days == df_missing.shape[0]

True

***Comment: Missing values over the entire period 1945-04-25--1945-11-05***

### imputation

In [11]:
# reference years are all years 5 years before and  5 years after 
reference_years = list(range(-5,6))
reference_years.remove(0)

# function for clean temperature values
def clean_temp(df,x):
    """
    returns a temperaure column in celcius with missing values imputed;
    imputation is done with the average of the temperautes on the same
    day over all the reference years; division by 10 for celcius value
    """
    # if missing value occurs
    if x['Q_TG']==9:
        
        # list reference dates
        reference_dates = [x['DATE']+relativedelta(years=y) for y in reference_years]
        
        # mean temperatue over the references dates
        temp_value = df[df['DATE'].isin(reference_dates)]['TG'].mean()
        
        # division by 10 to convert to celcius value
        return int(temp_value)/10
    
    # else just division by 10 to convert to celcius value
    return x['TG']/10

In [12]:
# create clean temperatutre column
df['TEMP'] = df.apply(
    lambda x: clean_temp(df,x),
    axis=1
)

In [13]:
# filter dataframe for days with missing value
df_missing = df[df['Q_TG']==9]

In [27]:
df_missing.head()

,SOUID,DATE,TG,Q_TG,TEMP
25316,127488,1945-04-25,-9999,9,11.2
25317,127488,1945-04-26,-9999,9,10.8
25318,127488,1945-04-27,-9999,9,10.3
25319,127488,1945-04-28,-9999,9,10.2
25320,127488,1945-04-29,-9999,9,9.7


In [28]:
df_missing.tail()

,SOUID,DATE,TG,Q_TG,TEMP
25506,127488,1945-11-01,-9999,9,5.2
25507,127488,1945-11-02,-9999,9,5.0
25508,127488,1945-11-03,-9999,9,6.1
25509,127488,1945-11-04,-9999,9,5.5
25510,127488,1945-11-05,-9999,9,4.5


### Train-Test Split

In [14]:
# lowercase column name
df.columns = df.columns.str.lower()

# extract relevant time series
clean_df = df.set_index('date')[['temp']]

In [15]:
clean_df.head()

,temp
date,
1876-01-01,2.2
1876-01-02,2.5
1876-01-03,0.3
1876-01-04,-5.8
1876-01-05,-9.8


In [16]:
# train data until Dec 31, 2021
train = clean_df[:'2021-12-31']

# test data for Jan 01-Dec 31, 2022
test = clean_df['2022-01-01':]

In [32]:
# shape of dataframes
train.shape, test.shape

((53326, 1), (365, 1))

In [33]:
# check train data
train.head()

,temp
date,
1876-01-01,2.2
1876-01-02,2.5
1876-01-03,0.3
1876-01-04,-5.8
1876-01-05,-9.8


In [18]:
# check test data
test.head()

,temp
date,
2022-01-01,12.0
2022-01-02,9.9
2022-01-03,8.9
2022-01-04,6.4
2022-01-05,3.8


In [17]:
# save train data as csv
train.to_csv('data/train.csv')

# save test data as csv
test.to_csv('data/test.csv')